# COSMOS master catalogue
## Preparation of Canada France Hawaii Telescope WIRDS Survey (CFHT-WIRDS) data

The catalogue is in `dmu0_CFHT-WIRDS`.

In the catalogue, we keep:

- The position;
- The stellarity;
- The aperture magnitude (3 arcsec).
- The total magnitude (Kron like aperture magnitude).



In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "wirds_ra"
DEC_COL = "wirds_dec"

## I - Column selection

In [ ]:
#We have to import and combine the H, J and Ks catalogues separately

epoch = 2007

imported_columns_h = OrderedDict({
        'NUMBER': "wirds_id"
        'ALPHA_J2000': "wirds_ra",
        'DELTA_J2000': "wirds_dec",
        'CLASS_STAR':  "wirds_stellarity",
        'MAG_APER': "m_ap_wirds_h",
        'MAGERR_APER': "merr_ap_wirds_h",
        'MAG_AUTO': "m_wirds_h",
        'MAGERR_AUTO': "merr_wirds_h",
        'FLUX_APER': "f_ap_wirds_h",
        'FLUXERR_APER': "ferr_ap_wirds_h",
        'FLUX_AUTO': "f_wirds_h",
        'FLUXERR_AUTO': "ferr_wirds_h"
        
    })


catalogue_h = Table.read("../../dmu0/dmu0_CFHT-WIRDS/data/COSMOS_H.fits")[list(imported_columns)]
for column in imported_columns_h:
    catalogue[column].name = imported_columns[column]
# Clean table metadata
catalogue_h.meta = None



imported_columns_j = OrderedDict({
        'NUMBER': "wirds_id"
        'ALPHA_J2000': "wirds_ra",
        'DELTA_J2000': "wirds_dec",
        'CLASS_STAR':  "wirds_stellarity",
        'MAG_APER': "m_ap_wirds_j",
        'MAGERR_APER': "merr_ap_wirds_j",
        'MAG_AUTO': "m_wirds_j",
        'MAGERR_AUTO': "merr_wirds_j",
        'FLUX_APER': "f_ap_wirds_j",
        'FLUXERR_APER': "ferr_ap_wirds_j",
        'FLUX_AUTO': "f_wirds_j",
        'FLUXERR_AUTO': "ferr_wirds_j"
        
    })


catalogue_j = Table.read("../../dmu0/dmu0_CFHT-WIRDS/data/COSMOS_J.fits")[list(imported_columns)]
for column in imported_columns_j:
    catalogue[column].name = imported_columns[column]
# Clean table metadata
catalogue_j.meta = None



imported_columns_ks = OrderedDict({
        'NUMBER': "wirds_id"
        'ALPHA_J2000': "wirds_ra",
        'DELTA_J2000': "wirds_dec",
        'CLASS_STAR':  "wirds_stellarity",
        'MAG_APER': "m_ap_wirds_ks",
        'MAGERR_APER': "merr_ap_wirds_ks",
        'MAG_AUTO': "m_wirds_ks",
        'MAGERR_AUTO': "merr_wirds_ks",
        'FLUX_APER': "f_ap_wirds_ks",
        'FLUXERR_APER': "ferr_ap_wirds_ks",
        'FLUX_AUTO': "f_wirds_ks",
        'FLUXERR_AUTO': "ferr_wirds_ks"
        
    })


catalogue_ks = Table.read("../../dmu0/dmu0_CFHT-WIRDS/data/COSMOS_Ks.fits")[list(imported_columns)]
for column in imported_columns_ks:
    catalogue[column].name = imported_columns[column]
# Clean table metadata
catalogue_ks.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        
        #catalogue[col][catalogue[col] <= 0] = np.nan
        #catalogue[errcol][catalogue[errcol] <= 0] = np.nan  
        

        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_cfhtls_u',
            'merr_ap_cfhtls_g',
            'merr_ap_cfhtls_r',
            'merr_ap_cfhtls_i',
            'merr_ap_cfhtls_z',]
FLAG_NAME = 'cfhtls_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_COSMOS.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "cfhtls_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/CFHTLS.fits".format(OUT_DIR), overwrite=True)